In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize Spark session
spark = SparkSession.builder.appName("data_insertion").getOrCreate()

# Sample data for the Customer table
customer_data = [
    Row(customer_id=101, customer_name="Alice"),
    Row(customer_id=102, customer_name="Bob"),
    Row(customer_id=103, customer_name="Charlie"),
]

# Create a DataFrame for the Customer table
customer_df = spark.createDataFrame(customer_data)

# Register the DataFrame as a temporary table
customer_df.createOrReplaceTempView("Customer")

# Sample data for the Orders table
orders_data = [
    Row(order_id=1, sale_date="2020-03-01", order_cost=1500, customer_id=101, seller_id=1),
    Row(order_id=2, sale_date="2020-05-25", order_cost=2400, customer_id=102, seller_id=2),
    Row(order_id=3, sale_date="2019-05-25", order_cost=800, customer_id=101, seller_id=3),
    Row(order_id=4, sale_date="2020-09-13", order_cost=1000, customer_id=103, seller_id=2),
    Row(order_id=5, sale_date="2019-02-11", order_cost=700, customer_id=101, seller_id=2),
]

# Create a DataFrame for the Orders table
orders_df = spark.createDataFrame(orders_data)

# Register the DataFrame as a temporary table
orders_df.createOrReplaceTempView("Orders")

# Sample data for the Seller table
seller_data = [
    Row(seller_id=1, seller_name="Daniel"),
    Row(seller_id=2, seller_name="Elizabeth"),
    Row(seller_id=3, seller_name="Frank"),
]

# Create a DataFrame for the Seller table
seller_df = spark.createDataFrame(seller_data)

# Register the DataFrame as a temporary table
seller_df.createOrReplaceTempView("Seller")

# Stop the Spark session
#spark.stop()


In [0]:
customer_df.display()

customer_id,customer_name
101,Alice
102,Bob
103,Charlie


In [0]:
orders_df.display()

order_id,sale_date,order_cost,customer_id,seller_id
1,2020-03-01,1500,101,1
2,2020-05-25,2400,102,2
3,2019-05-25,800,101,3
4,2020-09-13,1000,103,2
5,2019-02-11,700,101,2


In [0]:
seller_df.display()

seller_id,seller_name
1,Daniel
2,Elizabeth
3,Frank


Report the names of all sellers who did not make any sales in 2020.

Return the result table ordered by seller_name in ascending order.


In [0]:
from pyspark.sql.functions import year

In [0]:
orders_df_2020 = orders_df.filter(year(orders_df["sale_date"])==2020)

In [0]:
# Get a list of seller_ids that had sales in 2020
sellers_with_sales_2020 = orders_df_2020.select("seller_id").distinct()

In [0]:
sellers_not_in_2020 = seller_df.join(
    sellers_with_sales_2020,
    seller_df["seller_id"] ==orders_df_2020["seller_id"],
    "leftanti"
)

In [0]:
# Select and order the seller names
result = sellers_not_in_2020.select("seller_name").orderBy("seller_name")

# Show the result
result.show()

+-----------+
|seller_name|
+-----------+
|      Frank|
+-----------+



In [0]:
# Register the DataFrames as temporary tables
customer_df.createOrReplaceTempView("Customer")
orders_df.createOrReplaceTempView("Orders")
seller_df.createOrReplaceTempView("Seller")

In [0]:

query = """
SELECT seller_name
FROM Seller
WHERE seller_id NOT IN (
    SELECT seller_id
    FROM Orders
    WHERE YEAR(sale_date) = 2020
)
ORDER BY seller_name ASC
"""


result = spark.sql(query)


result.show()




+-----------+
|seller_name|
+-----------+
|      Frank|
+-----------+



In [0]:
%sql

SELECT seller_name
FROM Seller
WHERE seller_id NOT IN (
    SELECT seller_id
    FROM Orders
    WHERE YEAR(sale_date) = 2020
)
ORDER BY seller_name ASC

seller_name
Frank
